In [1]:
from src.services.llm import LLM
from src.services.logical_tree import LogicalTree
from src.utils.build_agent import query_based_on_tool_calls, fix_github_error
from typing import Callable, List, Dict, Any
from src.services.agents import Tool_Calls
from webui import launch_web

# from src.services.agents import Agent, Tool
from src.services.custom_tools import custom_tools, clean_training_logs
import glob
import os

os.environ["NO_PROXY"] = "*"

# llm = LLM(model_name="qwen3:8b")

llm = LLM(
    model_name="deepseek-chat",
    llm_url="https://api.deepseek.com/chat/completions",
    api_key="sk-2332c3d16a8d4f4ba1b3503074ba04c5",
    format="openai",
)
# llm = LLM(
#     model_name="gpt-5-2025-08-07",
#     llm_url="https://api.aimlapi.com/v1/chat/completions",
#     api_key="9ce046a9681446c48427b3fe4dd7cdd4",
#     format="openai",
# )
# llm = LLM(
#     model_name="gpt-4o",
#     llm_url="https://api.aimlapi.com/v1/chat/completions",
#     api_key="9ce046a9681446c48427b3fe4dd7cdd4",
#     format="openai",
# )



ct = custom_tools(
    HOST_DIR="/Users/yuanwen/Desktop/Docker_Environment/intern2/3/agent/issue_fix",
    HOST_DATA_DIR="/Users/yuanwen/Desktop/Docker_Environment/intern2/3/data/issue_fix",
    MAIN_DIR="/workspace",
    MAIN_DATA_DIR="/data",
    PYTHON_PATH="python",
    MATLAB_PATH="/Applications/MATLAB_R2023b.app/bin/matlab",
    ENV_NAME="fix_github_issue",
    LOCAL_TMP_PATH="/Users/yuanwen/Desktop/Docker_Environment/intern2/3/tmp",
    REMOTE_TMP_PATH="/tmp",
    BASE_ENV="python:3.7",
    PMC_URL="https://pmc.ncbi.nlm.nih.gov/articles/PMC7567795",
    llm=llm,
)
from src.services.llm import load_messages, save_messages
tool_calls_path = "/Users/yuanwen/Desktop/Docker_Environment/intern2/3/code/tool_calls_path.json"
env_tool_calls_path = "/workspace/tool_calls_path.json"

from datasets import load_from_disk
ds = load_from_disk("/Users/yuanwen/Desktop/Docker_Environment/intern2/4/data/SWE-bench_Lite")
print(ds)
sample = ds[0]
package_name = "package"

tc = Tool_Calls(PATH=tool_calls_path, ENV_PATH=env_tool_calls_path, MAX_CHAR=50000)
tc.clear()

launch_web()
message = fix_github_error(
    sample=sample,
    ct=ct,
    max_steps=10000,
    tc=tc,
    tree_filepath="/Users/yuanwen/Desktop/Docker_Environment/intern2/3/code/logical_tree_fix_issue.json",
    package_name=package_name,
    verbose=True,
    whether_recreate=False,
    save_filepath="/workspace/fix_issue.txt"
)

Dataset({
    features: ['repo', 'instance_id', 'base_commit', 'patch', 'test_patch', 'problem_statement', 'hints_text', 'created_at', 'version', 'FAIL_TO_PASS', 'PASS_TO_PASS', 'environment_setup_commit'],
    num_rows: 300
})
🌐 Gradio WebUI running at http://127.0.0.1:17860

[WebUI INFO]
  • Cloning GitHub repository...

Running on local URL:  http://0.0.0.0:17860

To create a public link, set `share=True` in `launch()`.



[INFO] Starting LLM query for goal 1...


[INFO] === Prompt step 1 ===

[INFO] Recomputed 0 nodes in total (excluding layer 0).

[LLM] Query with Tools
  [INPUT]
    You are solving a task step by step using a logical tree.
    
    - The **current node** you are working on is:
      - Name: fix github issue
      - Description: You are a professional open-source software engineer and bug-fixing agent.
    
    Your task:
    Given a real GitHub issue description from the SWE-bench dataset, produce the minimal code modification that fixes the described problem.
    
    ### Requirements
    1. Read the issue carefully and identify the root cause.
    2. Write ONLY a valid GitHub unified diff patch (git diff format) that would fix the bug.
    3. Do NOT include any explanations, comments, or Markdown formatting.
    4. The diff must start with the line:
       diff --git a/<path> b/<path>
    5. Use proper context lines and +/- markers so that the patch can be directly applied with `gi

KeyboardInterrupt: 

In [6]:
ds[100]

{'repo': 'django/django',
 'instance_id': 'django__django-15819',
 'base_commit': '877c800f255ccaa7abde1fb944de45d1616f5cc9',
 'patch': 'diff --git a/django/core/management/commands/inspectdb.py b/django/core/management/commands/inspectdb.py\n--- a/django/core/management/commands/inspectdb.py\n+++ b/django/core/management/commands/inspectdb.py\n@@ -127,12 +127,14 @@ def table2model(table_name):\n                     yield "# The error was: %s" % e\n                     continue\n \n+                model_name = table2model(table_name)\n                 yield ""\n                 yield ""\n-                yield "class %s(models.Model):" % table2model(table_name)\n-                known_models.append(table2model(table_name))\n+                yield "class %s(models.Model):" % model_name\n+                known_models.append(model_name)\n                 used_column_names = []  # Holds column names used in the table so far\n                 column_to_field_name = {}  # Maps column names 